# Benchmark Stratus Data Rates

We're interested in seeing how long it takes to create dynamically generated slices from Zarr stores located on different storage devices, namely Glade, Stratus, and AWS S3.  

The slices might be used for producing plots in interactive web applications run from NCAR web servers.

In [1]:
import xarray as xr
import dask

import fsspec
import s3fs

#from ncar_jobqueue import NCARCluster
from dask_jobqueue import PBSCluster
from distributed import Client

## Create and Connect to a Dask Distributed Cluster

### Casper Cluster Configuration

In [2]:
num_jobs = 35 #30 #10 #2 
walltime = "0:20:00"
memory='5GB' 

#cluster = NCARCluster(walltime=walltime, memory=memory)
cluster = PBSCluster(cores=1, processes=1, walltime=walltime, memory=memory, queue='casper', 
                     resource_spec='select=1:ncpus=1:mem=10GB',)
cluster.scale(jobs=num_jobs)

client = Client(cluster)
cluster

In [ ]:
# Use this to bail out if needed

cluster.close()

### Starting Point:  Load max daily temperatures from CESM LENS Data

#### Stratus as data source

In [3]:
s3_path = 's3://ncar-cesm-lens/atm/daily/cesmLE-RCP85-TREFHTMX.zarr' 
ds_stratus = xr.open_zarr(fsspec.get_mapper(s3_path, anon=True),
                  consolidated=True)

ds_stratus

<xarray.Dataset>
Dimensions:    (member_id: 40, time: 34675, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 1 2 3 4 5 6 7 8 ... 34 35 101 102 103 104 105
  * time       (time) object 2006-01-01 12:00:00 ... 2100-12-31 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(17338, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    TREFHTMX   (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 576, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.0
    NCO:                       4.4.2
    Version:                   $Name$
    host:                      tcs-f02n07
    important_note:            This data is part of the project 'Blind Evalua...
    initial_file:              b.e11.B20TRC5CNBDRD.f09_g16.105.cam.i.2006-01-...
    logname:                   mudryk
    nco_openmp_thread_number:  1
    revision_Id:               $Id$
    source:                    CAM
    title:                     UNSET
    topography_file:           /scratch/p/pjk/mudryk/cesm1_1_2_LENS/inputdata...

#### AWS as data source

In [4]:
# Connect to AWS S3 storage
fs = s3fs.S3FileSystem(anon=True)

# create a MutableMapping from a store URL
mapper = fs.get_mapper(s3_path)

# make sure to specify that metadata is consolidated
ds_aws = xr.open_zarr(mapper, consolidated=True)
ds_aws

<xarray.Dataset>
Dimensions:    (member_id: 40, time: 34675, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 1 2 3 4 5 6 7 8 ... 34 35 101 102 103 104 105
  * time       (time) object 2006-01-01 12:00:00 ... 2100-12-31 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(17338, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    TREFHTMX   (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 576, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.0
    NCO:                       4.4.2
    Version:                   $Name$
    host:                      tcs-f02n07
    important_note:            This data is part of the project 'Blind Evalua...
    initial_file:              b.e11.B20TRC5CNBDRD.f09_g16.105.cam.i.2006-01-...
    logname:                   mudryk
    nco_openmp_thread_number:  1
    revision_Id:               $Id$
    source:                    CAM
    title:                     UNSET
    topography_file:           /scratch/p/pjk/mudryk/cesm1_1_2_LENS/inputdata...

#### Glade as data source

In [5]:
zarr_folder = '/glade/scratch/bonnland/stratus-test/cesmLE-RCP85-TREFHTMX.zarr'

ds_glade = xr.open_zarr(zarr_folder, consolidated=True)
ds_glade

<xarray.Dataset>
Dimensions:    (member_id: 40, time: 34675, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 1 2 3 4 5 6 7 8 ... 34 35 101 102 103 104 105
  * time       (time) object 2006-01-01 12:00:00 ... 2100-12-31 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(17338, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    TREFHTMX   (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 576, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.0
    NCO:                       4.4.2
    Version:                   $Name$
    host:                      tcs-f02n07
    important_note:            This data is part of the project 'Blind Evalua...
    initial_file:              b.e11.B20TRC5CNBDRD.f09_g16.105.cam.i.2006-01-...
    logname:                   mudryk
    nco_openmp_thread_number:  1
    revision_Id:               $Id$
    source:                    CAM
    title:                     UNSET
    topography_file:           /scratch/p/pjk/mudryk/cesm1_1_2_LENS/inputdata...

In [6]:
def saveSlice(ds, start_year, end_year):
    '''Create and save time series for grid cell containing Boulder, Colorado.
    ''' 
    (lat, lon) = (40.01, 244.8)
    temps = ds.sel(lat=lat, lon=lon, method='nearest')
    temps = temps.sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))
    
    df = temps.TREFHTMX.to_dataframe()
    df.to_csv('Boulder_2010.csv', index=True)

### Choose slice parameters

In [7]:
start_year = 2007
#end_year = 2007
#end_year = 2017
end_year = 2057

###  Perform Benchmark Comparisons

In [8]:
%%time

# Test Glade performance
saveSlice(ds_glade, start_year, end_year)

CPU times: user 6.02 s, sys: 206 ms, total: 6.22 s
Wall time: 15.1 s


In [9]:
%%time

# Test Stratus performance
saveSlice(ds_stratus, start_year, end_year)

CPU times: user 8.24 s, sys: 315 ms, total: 8.55 s
Wall time: 58.3 s


In [10]:
%%time

# Test AWS S3 performance
saveSlice(ds_aws, start_year, end_year)

CPU times: user 7.74 s, sys: 322 ms, total: 8.06 s
Wall time: 48.1 s


In [ ]:
cluster.close()